## prophet
- 주요 구성요소 : Trend, Seasonality, Holiday
- Trend를 구성하는 g(t)는 주기적이지 않은 변화인 트렌드를 나타낸다.
- 부분적으로 선형 또는 logistic곡선으로 이루어져 있다.
- Seasonality를 나타내는 s(t)는 weekly, yearly 등 주기적으로 나타나는 패턴을 나타낸다.
- Holiday를 나타내는 h(t)는 휴일과 같이 불규칙한 이벤트를 나타낸다.
- 마지막 Ei는 정규분포를 따르는 오차이다.

In [ ]:
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = RuntimeWarning)
import datetime as dt
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
#import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import pandas as pd
from plotnine import *
import io
#to predictive A
import statistics as st
%matplotlib inline
import matplotlib.dates as mdates
from pandas import DataFrame
from pandas import to_datetime
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from  sklearn.metrics import mean_absolute_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 준비

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/BDA_CSV/sales_data.csv',encoding='unicode_escape')

In [ ]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])

In [ ]:
df_1  = df.copy()
df_1=df_1[df_1.Quantity>0]
df_1=df_1[df_1.UnitPrice>0]
df_1['ttp']= df_1['Quantity'] * df_1['UnitPrice']

In [ ]:
df_psp=df_1[['InvoiceDate','ttp']]
df_psp.rename(columns={'InvoiceDate':'ds','ttp':'y'},inplace=True)
df_psp=df_psp.set_index('ds')
df_psp

,y
ds,
2010-12-01 08:26:00,15.30
2010-12-01 08:26:00,20.34
2010-12-01 08:26:00,22.00
2010-12-01 08:26:00,20.34
2010-12-01 08:26:00,20.34
...,...
2011-12-09 12:50:00,10.20
2011-12-09 12:50:00,12.60
2011-12-09 12:50:00,16.60


In [ ]:
df_d = df_psp.resample('D')['y'].sum()
df_m = df_psp.resample('M')['y'].sum()
df_y = df_psp.resample('Y')['y'].sum()
df_d=df_d.to_frame()
df_m=df_m.to_frame()
df_y=df_y.to_frame()

In [ ]:
df_d.reset_index(inplace=True)
df_m.reset_index(inplace=True)
df_y.reset_index(inplace=True)

In [ ]:
df_d

,ds,y
0,2010-12-01,58960.79
1,2010-12-02,47748.38
2,2010-12-03,46943.71
3,2010-12-04,0.00
4,2010-12-05,31774.95
...,...,...
369,2011-12-05,88741.96
370,2011-12-06,56713.21
371,2011-12-07,75439.16
372,2011-12-08,82495.00


In [ ]:
train=df_d[:334]
test = df_d[334:]

## Trend
- changepoints : auto [default]

  변경점을 수동으로 설정하기 위한 파라미터


- changepoint_prior_scale : 0.05 [default]

  추세에 따라 데이터를 유연하게 반열할 건지 추세에 유연하지 않게 반영할 것인지 결정.

  추세에 덜 유연하게 하려면 0.05보다 작게 설정 / 유연하게 반응하려면 0.05보다 크게 설정

  추가적으로 기준일 전날까지 반영할 경우 lower_window = -1, upper_window = 0 / 기준일 다음날까지 반영하면 lower_window = 0, upper_window = 1
- n_changepoints :  [default]

  changepoint의 개수 설정
- changepoint_range : 0.8 [default]

  changepoint설정 가능 범위.(기본적으로 데이터 중 80% 범위 내에서 changepoint를 설정)

In [ ]:
prophet = Prophet(changepoints = auto, changepoint_prior_scale = 0.5,daily_seasonality=True)

NameError: ignored

## Seasonality
- yearly_seasonality : auto [default]

  데이터가 연간 계절성을 가지면 True 아니면 False로 설정

- weekly_seasonality
- daily_seasonality
- seasonality_prior_scale :  [default]

  계절의 유연성을 제어. 값이 클수록 계절성의 영향이 커짐.

  연간/주간/일별 계절성과 관련이 깊다.

  add_seasonality와 같이 계절성을 임의로 추가 가능.
- seasonality_mode : additive [default]

  옵션 : ['additive', 'multiflicative']

  additive : 시계열 데이터가 진폭이 일정할 때 사용

  multiflicate : 시계열 데이터의 진폭이 점점 증가하거나 감소할 때 사용

## Holiday
- holidays

  지정된 공휴일의 데이터 프레임을 전달하기 위해 사용
- holiday_prior_scale : 10.0 [default]

  휴일의 영향에 따라 유연성을 조절

  add_country_holidays(country_name = '')로 원하는 국가의 휴일 반영 가능

## 기타
- interval_width : 0.8 [default]

  추세 예측 정확도 구간범위, 미래에 얼마나 변동할 것인지 설정

- growth : linear [default]

  옵션 : ['linear', 'logistic']

